In [20]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import math
import numpy as np

ROW_LENGTH = 62
COLUMN_LENGTH = 1.25
NUMBER_OF_ROWS = 8

In [21]:
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_temp.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_rogers_gibbs.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_python.pickle')
print document_topic_probabilities[0:10]
document_ids_map = pd.read_pickle('../pickles/doc_id_mapping.pickle').to_dict()
inverted_document_ids_map = {v: int(k) for k, v in document_ids_map.iteritems()}

[[ 0.04117647  0.00196078  0.00196078  0.57058824  0.02156863  0.08039216
   0.02156863  0.00196078  0.19803922  0.06078431]
 [ 0.00625     0.00625     0.00625     0.00625     0.00625     0.00625
   0.56875     0.00625     0.38125     0.00625   ]
 [ 0.00666667  0.07333333  0.00666667  0.00666667  0.00666667  0.00666667
   0.74        0.00666667  0.14        0.00666667]
 [ 0.00714286  0.00714286  0.00714286  0.00714286  0.00714286  0.00714286
   0.00714286  0.00714286  0.93571429  0.00714286]
 [ 0.00666667  0.00666667  0.00666667  0.00666667  0.00666667  0.07333333
   0.80666667  0.00666667  0.07333333  0.00666667]
 [ 0.00666667  0.00666667  0.00666667  0.00666667  0.00666667  0.00666667
   0.00666667  0.00666667  0.94        0.00666667]
 [ 0.00714286  0.00714286  0.00714286  0.00714286  0.15        0.00714286
   0.07857143  0.00714286  0.72142857  0.00714286]
 [ 0.00666667  0.00666667  0.00666667  0.00666667  0.00666667  0.00666667
   0.14        0.00666667  0.80666667  0.00666667]
 [ 

In [22]:
def calculate_the_sizes(data, number_of_rows, row_length, column_length):
    data_size = len(data)
    single_scan_size = (data_size + 1) / number_of_rows
    column_size = int(math.floor(single_scan_size \
    * (column_length / (row_length + column_length))))
    row_size = single_scan_size - column_size 
    return data_size, single_scan_size, column_size, row_size

In [23]:
def generate_coordinates(data_size, column_size, row_size):    
    x_coord = 0
    y_coord = NUMBER_OF_ROWS - 1
    row_scan = True
    scanning_right = True
    coordinates = {}
    for j in range(0, NUMBER_OF_ROWS):
        for i in range(0, row_size):
            doc_id = i + column_size * j + row_size * j
            coordinates[doc_id] = {'x': x_coord, 'y': y_coord}
            # Switching horizontal scanning to vertical.
            if (i % (row_size - 1) == 0 and i != 0):
                y_coord -= 1
                continue
            if scanning_right:
                x_coord += 1
            else:
                x_coord -= 1         
        # Reversing the scanner's direction.
        scanning_right = not scanning_right
    return coordinates

In [24]:
data_size, single_scan_size, column_size, row_size = \
    calculate_the_sizes(inverted_document_ids_map, NUMBER_OF_ROWS, ROW_LENGTH, COLUMN_LENGTH)
coordinates_mapping = generate_coordinates(data_size, column_size, row_size)

In [25]:
topic_id = 3

probability_matrix = np.zeros(shape=(NUMBER_OF_ROWS, row_size))
for i, doc in enumerate(document_topic_probabilities):
    doc_id = inverted_document_ids_map[i]
#     doc_id = i
    if doc_id in coordinates_mapping:
        doc_coord = coordinates_mapping[doc_id]
        probability_matrix[doc_coord['y']][doc_coord['x']] = doc[topic_id]

In [26]:
import plotly.plotly as py
import plotly.graph_objs as go

x_vector = np.zeros(shape=(row_size, 1))
for i in range(0, row_size):
    x_vector[i] = i
y_vector = np.zeros(shape=(NUMBER_OF_ROWS, 1))
for i in range(0, NUMBER_OF_ROWS):
    y_vector[i] = i

layout = go.Layout(
    title='\'Topic %s\' probability distribution.' % topic_id,
    height=350,
    xaxis=dict(title='x'),
    yaxis=dict(title='y')
)
data = [
    go.Heatmap(
        z=probability_matrix,
        x=x_vector,
        y=y_vector,
        colorscale='Electric',
        colorbar=dict(title='Likelihood')
    )
]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='topic_%s' % topic_id)